In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage

colors = [
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'blue','blue','blue',
'red','red','red',
'green','green','green',
'cyan','cyan','cyan',
'orange','orange','orange',
    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']

Autosaving every 180 seconds


In [40]:
# name of .mat file containing trigger poitns
import scipy.io as sio

fname_in = '/media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-2020  7_17 PM (manual)CallBurstTimes.mat'
#data = hdf5storage.read(filename=fname_in) 
data = sio.loadmat(fname_in) 
#print (data)

BurstCall = data['BurstCall']
BurstTS = data['BurstTS']
durations = data['durations']
n_per_burst = data['n_per_burst']

print ("BurstTS: ", BurstTS)
# video name
video_name = "/media/cat/1TB/insync_cm5635/march_2/video/completed/dan_USV_triggered/2020-3-15_03_51_56_415333/2020-3-15_03:51:56:415333_compressed.avi"
frame_rate = 25

# load frame timestamps and set them to start at 0sec
times_loaded = np.loadtxt('/media/cat/1TB/insync_cm5635/march_2/times/2020-3-15_03:51:56:415333.txt')
times_loaded = times_loaded-times_loaded[0]
print ("zero centred: ", times_loaded)



BurstTS:  [[ 133.36347057]
 [ 594.78355127]
 [ 628.70578627]
 [ 676.59002458]
 [ 677.26379442]
 [ 689.57244896]
 [ 690.7412301 ]
 [ 696.1685718 ]
 [ 696.86554853]
 [1373.93435597]]
zero centred:  [0.00000000e+00 4.00029998e-02 8.00069999e-02 ... 3.59973906e+03
 3.59977907e+03 3.59981907e+03]


In [41]:
# load .mat data structures.

# Rate = data[0][0][0][0][0][4][0][0]
# Box = data[0][0][0][0][0][4][0][1]
# ReBox = data[0][0][0][0][0][4][0][2]
# Score = data[0][0][0][0][0][4][0][3]
# Audio = data[0][0][0][0][0][4][0][4]
# Type = data[0][0][0][0][0][4][0][5]
# Power = data[0][0][0][0][0][4][0][6]
# Accept = data[0][0][0][0][0][4][0][7]
# print (data[0][0][0][0][0][9])

start = BurstTS
print ("USV times: ", start)

#np.save(fname_in[:-4]+"_USV_starts_long_rat.npy", start)



USV times:  [[ 133.36347057]
 [ 594.78355127]
 [ 628.70578627]
 [ 676.59002458]
 [ 677.26379442]
 [ 689.57244896]
 [ 690.7412301 ]
 [ 696.1685718 ]
 [ 696.86554853]
 [1373.93435597]]


In [42]:
# video writing step
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
original_vid = cv2.VideoCapture(video_name)

ctr=0
for time_ in start:

    # figure out index of starting frame and move video to location
    # assume 25.0 hz was perect
    frame_no1 = int(time_*frame_rate)
    
    # use timestamp file
    idx = np.where(times_loaded>time_)[0][0]
    frame_no2 = idx

    frame_used = frame_no2
    
    print ("loading USV "+str(ctr)+ " , at time: ", time_, " , frame # (based on 25.0hz): ", frame_no1, ", timestamp frame: ", frame_no2)
    print ("setting frame time to: ", frame_used-frame_rate*3)
    original_vid.set(1,frame_used-frame_rate*3)

    # initialize video writer for file to be saved
    print (fname_in+'/audio_triggered/')
    print ("KKK: ", fname_in+'/')
    try:
        os.mkdir(fname_in[:-4]+'/')
    except:
        pass
    print ("time: ", time_)
    fname_out = fname_in[:-4]+'/'+str(round(time_[0],3))+\
                'sec_USV_'+ str(ctr+1).zfill(3)+'.avi'
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (512,512), True)
    
    #Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
    for k in range(frame_rate*6):
        ret, frame = original_vid.read()
        
        frame_cropped = frame[:,150:1174][::2,::2]
        #frame_cropped[:7,:20]= digits[0]
        
        if k == frame_rate*3:
            frame_cropped[:25,:25]= (np.float32(
                matplotlib.colors.to_rgb('red'))*255.).astype('uint8')
        
        video_out.write(frame_cropped)

    ctr+=1
    
    video_out.release()
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    #break
    

loading USV 0 , at time:  [133.36347057]  , frame # (based on 25.0hz):  3334 , timestamp frame:  3334
setting frame time to:  3259
/media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-2020  7_17 PM (manual)CallBurstTimes.mat/audio_triggered/
KKK:  /media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-2020  7_17 PM (manual)CallBurstTimes.mat/
time:  [133.36347057]
loading USV 1 , at time:  [594.78355127]  , frame # (based on 25.0hz):  14869 , timestamp frame:  14869
setting frame time to:  14794
/media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-2020  7_17 PM (manual)CallBurstTimes.mat/audio_triggered/
KKK:  /media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-2020  7_17 PM (manual)CallBurstTimes.mat/
time:  [594.78355127]
loading USV 2 , at time:  [628.70578627]  , frame # (based on 25.0hz):  15717 , timestamp frame:  15717
setting frame time to:  15642
/media/cat/1TB/insync_cm5635/Bursts/2020-3-15_03_51_56_415333 Apr-21-

In [134]:
import pandas

df = pandas.read_excel('/media/cat/1TB/insync_cm5635/march_2/video/completed/2020-3-15_03:51:56:415333_compressed/2020-3-15_03_51_56_415333.xlsx')

# print the column names
print (df.columns)

# get the values for a given column
times_ = df[df.columns[0]].values
time_array = []
for t in times_:
    time_array.append((t.hour * 60 + t.minute) * 60 + t.second)

male = df[df.columns[1]].values
female = df[df.columns[2]].values
pup1 = df[df.columns[3]].values
pup2 = df[df.columns[4]].values

# 
idx = np.where(male=='A-C')[0]
male[idx]=13
idx = np.where(male=='A-B')[0]
male[idx]=13

idx = np.where(female=='B-D')[0]
female[idx]=13
idx = np.where(female=='B-C')[0]
female[idx]=13

idx = np.where(pup1=='B-C')[0]
pup1[idx]=13
idx = np.where(pup1=='C-D')[0]
pup1[idx]=13
idx = np.where(pup1=='A-C')[0]
pup1[idx]=13

idx = np.where(pup2=='A-C')[0]
pup2[idx]=13
idx = np.where(pup2=='A-B')[0]
pup2[idx]=13
idx = np.where(pup2=='C-D')[0]
pup2[idx]=13
idx = np.where(pup2=='A-D')[0]
pup2[idx]=13
idx = np.where(pup2=='B-D')[0]
pup2[idx]=13
#
plt.plot(male, c='blue', label="male")
plt.plot(female, c='red', label="female")
plt.plot(pup1, c='green', label="pup1")
plt.plot(pup2, c='cyan', label="pup2")
plt.xlabel("Time (sec)", fontsize=20)
plt.ylabel("States",fontsize=20)
plt.legend(fontsize=20)

plt.scatter(start, start*0+13, c='black')

labels = [
    "1 Scratching walls",
    "2 Digging beddings",
    "3 Playing (scratching) with blocks or hut",
    "4 Self grooming",
    "5 Sleeping",
    "6 Walking",
    "7 Chasing",
    "8 Being Chased",
    "9 Eating",
    "10 Drinking/ at spout",
    "11 Occluded",
    "12",
    "13 Social Interaction"
]
plt.yticks(np.arange(1,14,1),labels)



plt.show()


Index(['Unnamed: 0', 'A-Male adult', 'B-Female adult', 'C-fluffy pup',
       'D-shaved pup', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [78]:
# plot time course of video vs. other data 

fname_vid = '/media/cat/1TB/insync_cm5635/march_2/video/completed/dan_USV_triggered/2020-3-15_03_51_56_415333/2020-3-15_03:51:56:415333_compressed.avi'
original_vid.release()
original_vid = cv2.VideoCapture(video_name)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, 0)

ctr=0
motion = []
ret, old_frame = original_vid.read()
while True:
    if ctr%1000==0:
        print (ctr)

    ret, frame = original_vid.read()
    
    #motion.append([np.sum(old_frame[:,:,0]-frame[:,:,0]),
    #              np.sum(old_frame[:,:,1]-frame[:,:,1]),
    #              np.sum(old_frame[:,:,2]-frame[:,:,2])])
    motion.append(np.sum(old_frame[:,:,1]-frame[:,:,1]))
    
    if ret==False:
        break
    #if ctr>1000:
    #    break
    ctr+=1
    old_frame = frame
        
motion = np.vstack(motion)
print (motion)
print (motion.shape)
fig=plt.figure()
plt.plot(motion,c='green')
# plt.plot(motion[:,1],c='green')
# plt.plot(motion[:,2],c='blue')
plt.show()


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000


TypeError: 'NoneType' object is not subscriptable

In [81]:
print (np.array(motion).shape)

np.save('/media/cat/1TB/insync_cm5635/march_2/video/completed/2020-3-15_03:51:56:415333_compressed/motion.npy', motion)

(89987,)


In [84]:
fs=25

f, Pxx_den = signal.periodogram(np.float32(motion).squeeze(), fs)
print (f)
print (f.shape)
print (Pxx_den.shape)
plt.semilogy(f, Pxx_den)
#plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.title("Video power spectrum")
plt.show()

[0.00000000e+00 2.77817907e-04 5.55635814e-04 ... 1.24993055e+01
 1.24995833e+01 1.24998611e+01]
(44994,)
(44994,)


In [71]:
# filter data


(100000,)


In [77]:
import skvideo.io  
fname_vid = '/media/cat/1TB/insync_cm5635/march_2/video/completed/dan_USV_triggered/2020-3-15_03_51_56_415333/2020-3-15_03:51:56:415333_compressed.avi'
videodata = skvideo.io.vread(fname_vid)  
print(videodata.shape)

MemoryError: Unable to allocate 330. GiB for an array with shape (89988, 1024, 1280, 3) and data type uint8